In [ ]:
# set correct path for imports for this notebook
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# external imports 
import logging
logging.basicConfig(level=logging.INFO)

# local imports
from src.fetch.google_reviews import fetch_google_reviews 
from src.fetch.google_place import fetch_place_id, fetch_restaurant_info, Restaurant
from backend.src.nlp.extractor_local import extract_dishes, _cached_extract_single
from src.ranking.functions import assign_rankings
from src.recommendation.recs import form_recommendations

# Clear dish extractor cache to ensure fresh model load
_cached_extract_single.cache_clear()


INFO:src.nlp.dish_extractor:✅ Using 8 CPU threads for PyTorch


In [ ]:
r1_id = fetch_place_id("funkyfish berlin")
r2_id = fetch_place_id("standard pizza west berlin")
r3_id = fetch_place_id("borschert berlin")
r4_id = fetch_place_id("Gio's Berlin")

r4_id 

INFO:src.fetch.google_place:
 Found restaurant: FunkyFisch
Kantstraße 135-136, 10625 Berlin, Deutschland
ID = ChIJU7-KvONQqEcRCmPCCvo-Fg4

INFO:src.fetch.google_place:
 Found restaurant: Standard Serious Pizza Charlottenburg
Schlüterstraße 63, 10625 Berlin, Deutschland
ID = ChIJc2uwDI1RqEcRfAsy3YxyfIY

INFO:src.fetch.google_place:
 Found restaurant: Borchardt
Französische Str. 47, 10117 Berlin, Deutschland
ID = ChIJVfEmINtRqEcRsch0pslaanQ

INFO:src.fetch.google_place:
 Found restaurant: Gio's
Dresdener Str. 16, 10999 Berlin, Deutschland
ID = ChIJ65Liv5JPqEcRodr8aFFNAvE

INFO:src.fetch.google_place:
 Found restaurant: Paolo Pinkel
Karl-Marx-Straße 55, 12043 Berlin, Deutschland
ID = ChIJpRECXmtPqEcRJMfokCTNvD8



'ChIJ65Liv5JPqEcRodr8aFFNAvE'

In [3]:
r1 = Restaurant(r1_id)
r4 = Restaurant(r4_id)

In [4]:
r1_id

'ChIJU7-KvONQqEcRCmPCCvo-Fg4'

In [14]:
r1_reviews = fetch_google_reviews(r1_id)
r2_reviews = fetch_google_reviews(r2_id)
r3_reviews = fetch_google_reviews(r3_id)
r4_reviews = fetch_google_reviews(r4_id)

INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for FunkyFisch (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Standard Pizza West (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Borchardt (EN)
INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Gio's (EN)


In [ ]:
extract_dishes_openai(r4_reviews)

INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Paolo Pinkel (EN)
INFO:src.nlp.extractor_openai:🚀 Starting extraction for 5 text chunks...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:src.nlp.extractor_openai:✅ Completed dish extraction for 5 reviews in 4.75s


[{'id': None,
  'source': 'google',
  'author': 'Gorka Biern',
  'text': 'Good food, good music, good people! We had some problems with the order (they forgot to bring 2 dishes) but it was solved and we enjoyed the meal. Big plates and tasty! Nice cocktails too.',
  'dishes': [],
  'rating': 4,
  'date': 1747644232,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/103605384736928070620/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Deer Ozzie',
  'text': 'one of my favorite places in Neukölln.\nI have been so many times and every time I am 100% satisfied. I loved the ceviche, the roastbeef and the schnitzel! The waiters are also so friendly❤️',
  'dishes': [],
  'rating': 5,
  'date': 1749415038,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/116650280114912458563/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'M K',
  'text': 'The restaurant was absolutely packed when we arrived, b

In [ ]:
extract_dishes(r4_reviews, True)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:__main__:📝 Good food, good music, good people! We had some problems with the order (they forgot to bring 2 dishes) but it was solved and we enjoyed the meal. Big plates and tasty! Nice cocktails too.
INFO:__main__:🍽️ none

INFO:__main__:📝 one of my favorite places in Neukölln.
I have been so many times and every time I am 100% satisfied. I loved the ceviche, the roastbeef and the schnitzel! The waiters are also so friendly❤️
INFO:__main__:🍽️ schnitzel, roastbeef, ceviche

INFO:__main__:📝 The restaurant was absolutely packed when we arrived, but the staff managed to squ

[{'id': None,
  'source': 'google',
  'author': 'Gorka Biern',
  'text': 'Good food, good music, good people! We had some problems with the order (they forgot to bring 2 dishes) but it was solved and we enjoyed the meal. Big plates and tasty! Nice cocktails too.',
  'dishes': [],
  'rating': 4,
  'date': 1747644232,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/103605384736928070620/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Deer Ozzie',
  'text': 'one of my favorite places in Neukölln.\nI have been so many times and every time I am 100% satisfied. I loved the ceviche, the roastbeef and the schnitzel! The waiters are also so friendly❤️',
  'dishes': [{'name': 'schnitzel', 'ranking': None},
   {'name': 'roastbeef', 'ranking': None},
   {'name': 'ceviche', 'ranking': None}],
  'rating': 5,
  'date': 1749415038,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/116650280114912458563/reviews',
  'language': 'en'},

In [8]:
assign_rankings(r4_reviews, True)

Assigned ranking -1 to dish 'mini georgian bread' in review #1
Assigned ranking -1 to dish 'dessert' in review #1
Assigned ranking -1 to dish 'adjapsandali' in review #1
Assigned ranking -1 to dish 'dumplings' in review #4
Assigned ranking -1 to dish 'khachapuri imeruli' in review #5
Assigned ranking -1 to dish 'khinkali with meat' in review #5
Assigned ranking -1 to dish 'khinkali with cheese' in review #5


In [9]:
r4_reviews

[{'id': None,
  'source': 'google',
  'author': 'Nika Chugoshvili',
  'text': 'Friendly cozy and family atmosphere with authentic food. Not too pretty but original. Taste is priority. Mini georgian bread was super nice, adjapsandali and dessert as well.',
  'dishes': [{'name': 'mini georgian bread', 'ranking': -1},
   {'name': 'dessert', 'ranking': -1},
   {'name': 'adjapsandali', 'ranking': -1}],
  'rating': 5,
  'date': 1751374444,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/109702310661589209222/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Gaia Tassinari',
  'text': 'What a wonderful dining experience. Authentic delicious food, lovely service, amazing red wine. Thank you! Will be back for more.',
  'dishes': [],
  'rating': 5,
  'date': 1755459365,
  'url': None,
  'author_url': 'https://www.google.com/maps/contrib/113896279646697196969/reviews',
  'language': 'en'},
 {'id': None,
  'source': 'google',
  'author': 'Steven Ti

In [10]:
form_recommendations(r4_reviews)


[{'dish_name': 'mini georgian bread',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'dessert',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'adjapsandali',
  'ranking': -1,
  'author': 'Nika Chugoshvili',
  'source': 'google',
  'timestamp': 1751374444,
  'review_link': None},
 {'dish_name': 'dumplings',
  'ranking': -1,
  'author': 'Nick Orloff',
  'source': 'google',
  'timestamp': 1759490727,
  'review_link': None},
 {'dish_name': 'khachapuri imeruli',
  'ranking': -1,
  'author': 'Angelina Fedorenko',
  'source': 'google',
  'timestamp': 1742291482,
  'review_link': None},
 {'dish_name': 'khinkali with meat',
  'ranking': -1,
  'author': 'Angelina Fedorenko',
  'source': 'google',
  'timestamp': 1742291482,
  'review_link': None},
 {'dish_name': 'khinkali with cheese',
  'ranking': -1,
  'author': 'Ang

In [28]:
get_recommendations(r2_id)

INFO:src.fetch.google_reviews:✅ Retrieved 5 reviews for Standard Pizza West (EN)
INFO:__main__:📄 Retrieved 5 reviews
INFO:src.nlp.extractor_openai:🚀 Starting extraction for 5 text chunks...
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:src.nlp.extractor_openai:📝 A really good pizzeria even if not extraordinary.

The ingredients were of the highest quality and the dough was good to...
INFO:src.nlp.extractor_openai:🍽️ Extracted: none
INFO:src.nlp.extractor_openai:📝 Very good pizza (Gialla e Rossa). The crust may be a little overcooked but not burnt on the bottom (most important !). Q...
INFO:src.nlp.extractor_openai

Assigned ranking -1 to dish 'pizza' in review #2
Assigned ranking -1 to dish 'gialla e rossa' in review #2
Assigned ranking -1 to dish 'pizza' in review #3
Assigned ranking -1 to dish 'the pistachio tiramisu' in review #3
Assigned ranking -1 to dish 'neapolitan-style pizza' in review #4
Assigned ranking -1 to dish 'nduja' in review #5
Assigned ranking -1 to dish 'neapolitan-style pizza' in review #5
Assigned ranking -1 to dish 'carciofi' in review #5


{'recommendations': [{'dish_name': 'pizza',
   'ranking': -1,
   'author': 'K. “Dr. Slayer Von Pizza”',
   'source': 'google',
   'timestamp': 1755953081,
   'review_link': None},
  {'dish_name': 'gialla e rossa',
   'ranking': -1,
   'author': 'K. “Dr. Slayer Von Pizza”',
   'source': 'google',
   'timestamp': 1755953081,
   'review_link': None},
  {'dish_name': 'pizza',
   'ranking': -1,
   'author': 'o lee',
   'source': 'google',
   'timestamp': 1760896332,
   'review_link': None},
  {'dish_name': 'the pistachio tiramisu',
   'ranking': -1,
   'author': 'o lee',
   'source': 'google',
   'timestamp': 1760896332,
   'review_link': None},
  {'dish_name': 'neapolitan-style pizza',
   'ranking': -1,
   'author': 'Marfa Gulas',
   'source': 'google',
   'timestamp': 1734257472,
   'review_link': None},
  {'dish_name': 'nduja',
   'ranking': -1,
   'author': 'Lara Lima',
   'source': 'google',
   'timestamp': 1678434713,
   'review_link': None},
  {'dish_name': 'neapolitan-style pizza',
